# Lab 4. Media Operations Agent Collaborator

## Introduction

In this notebook we show you how to create multi-agent collaborator feature on Amazon Bedrock.

[Multi-agent Collaboration](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-multi-agents-collaboration.html) is a Amazon Bedrock Agents native capability that enables a hierarchical collaboration between agents. You can now enable agent collaboration and associate secondary agents to a supervisor one. These secondary agents can be any existing agent within the same account, including agents that have collaboration themselves. This composable pattern allows you to build a chain of agents, as shown in the figure below.

![collaboration_hierarch](static/multi-agent_flow.png)

In this lab, we will create a supervisor agent that interacts with the subagents created in the previous lab.

## Setup

Make sure that your boto3 version is the latest one.

In [221]:
!pip freeze | grep boto3

boto3==1.37.7


### Creating Agent

On this section we're going to declare global variables that will be act as helpers during entire notebook and you will start to create your agent.

In [222]:
import boto3

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
account_id_suffix = account_id[:3]
agent_suffix = f"{region}-{account_id_suffix}"

agent_foundation_model = [
    # 'us.amazon.nova-pro-v1:0',
    'us.anthropic.claude-3-5-sonnet-20241022-v2:0',
    'us.anthropic.claude-3-5-sonnet-20240620-v1:0',
    'anthropic.claude-3-sonnet-20240229-v1:0',
    'anthropic.claude-3-haiku-20240307-v1:0'
]

In [223]:
agent_name = f"su-agent-{agent_suffix}"

agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'

In [224]:
knowledge_base_name = f'{agent_name}'
knowledge_base_description = "This knowledge base contain guideline requirements and output schema for different type of video posts"

suffix = f"{region}-{account_id}"
bucket_name = f'{agent_name}-{suffix}'

bucket_name

'su-agent-us-west-2-241-us-west-2-241533163649'

### Importing helper functions

On following section, we're adding `bedrock_agent_helper.py` on Python path, so the files can be recognized and their functionalities can be invoked.

Now, you're going to import from helper classes `bedrock_agent_helper.py`.
 
Those files contain helper classes totally focused on make labs experience smoothly. 

All interactions with Bedrock will be handled by these classes.

Following are methods that you're going to invoke on this lab:

On `agents.py`:

- `create_agent`: Create a new agent and respective IAM roles
- `associate_agents`: Associate sub-agents with multi-agent collaborator

In [225]:
import sys
import uuid
import time
import json

sys.path.insert(0, '..')

from helper.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
from helper.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)
agents = AgentsForAmazonBedrock()
kb = KnowledgeBasesForAmazonBedrock()

Create the media ops agent

In [226]:
media_ops_agent = agents.create_agent(
    agent_name,
    """
        You are a media ops helper bot.
        You can help users analyze video and generate different type of post in compliance with organizational standards.
    """,
    """
        Your capabilities include:
        <capabilities>
        - Access to compliance documenation, can extract compliance requirements and output schema (DO NOT make up requirements)
        - create and enhance different type of social, website, and internal logging post for videos.
        - review and QC check post and create a list of violations base ONLY on the compliance documentation.
        - Access to different tools to detect key figure, lookup film and event information.
        </capabilities>

        Core behaviors:
        <behaviors>
        1. Always use available information systems before asking customers for additional details
        2. Maintain a professional yet conversational tone
        3. Provide clear, direct answers without referencing internal systems or data sources
        4. Present information in an easy-to-understand manner
        5. Resist the temptation to ask the user for input. Only do so after you have exhausted available actions. 
        6. Never ask the user for information that you already can retrieve yourself through available actions.
        </behaviors>
    """,
    agent_foundation_model,
    agent_collaboration='SUPERVISOR'
)

media_ops_agent

('AD9HMFOUHU',
 'TSTALIASID',
 'arn:aws:bedrock:us-west-2:241533163649:agent-alias/AD9HMFOUHU/TSTALIASID')

In [227]:
media_ops_agent_name = agent_name
media_ops_agent_id = media_ops_agent[0]
%store media_ops_agent_id
%store media_ops_agent_name

Stored 'media_ops_agent_id' (str)
Stored 'media_ops_agent_name' (str)


### Associate Collaborators

On this section, we're going to recover alias_id from previous agents (sub-agents) to add all of them inside energy one (which is multi-agent collaborator).

In [228]:
%store -r
syn_agent_alias_arn, qc_agent_alias_arn, film_agent_alias_arn, sport_agent_alias_arn

('arn:aws:bedrock:us-west-2:241533163649:agent-alias/Q35CY7LJXH/WDJVWHIQ3X',
 'arn:aws:bedrock:us-west-2:241533163649:agent-alias/3C2MWJKT0D/IZ9Y83APAP',
 'arn:aws:bedrock:us-west-2:241533163649:agent-alias/YMRXSPKGV0/C185HNE8PM',
 'arn:aws:bedrock:us-west-2:241533163649:agent-alias/MLCUVGISY6/HLRZTEQL3T')

In [229]:
sub_agents_list = [
    {
        'sub_agent_alias_arn': film_agent_alias_arn,
        'sub_agent_instruction': """Lookup film information like title, plotline, and etc. This agent can also identify cast member and their role from the film""",
        'sub_agent_association_name': 'FilmAgent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {
        'sub_agent_alias_arn': syn_agent_alias_arn,
        'sub_agent_instruction': """Create different types of video posts (Social Posts, Company Website, Internal Logs) or imporove existing post based on QC feedback.""",
        'sub_agent_association_name': 'SynopsisAgent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {
        'sub_agent_alias_arn': sport_agent_alias_arn,
        'sub_agent_instruction': """Lookup Sport event name or venu information and the athlete""",
        'sub_agent_association_name': 'SportAgent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {
        'sub_agent_alias_arn': qc_agent_alias_arn,
        'sub_agent_instruction': """Review different types of post against guidelines requirements and output schema, report any compliance violations.""",
        'sub_agent_association_name': 'QCAgent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    }
]

Check if parameters are correclty formated

In [230]:
sub_agents_list

[{'sub_agent_alias_arn': 'arn:aws:bedrock:us-west-2:241533163649:agent-alias/YMRXSPKGV0/C185HNE8PM',
  'sub_agent_instruction': 'Lookup film information like title, plotline, and etc. This agent can also identify cast member and their role from the film',
  'sub_agent_association_name': 'FilmAgent',
  'relay_conversation_history': 'TO_COLLABORATOR'},
 {'sub_agent_alias_arn': 'arn:aws:bedrock:us-west-2:241533163649:agent-alias/Q35CY7LJXH/WDJVWHIQ3X',
  'sub_agent_instruction': 'Create different types of video posts (Social Posts, Company Website, Internal Logs) or imporove existing post based on QC feedback.',
  'sub_agent_association_name': 'SynopsisAgent',
  'relay_conversation_history': 'TO_COLLABORATOR'},
 {'sub_agent_alias_arn': 'arn:aws:bedrock:us-west-2:241533163649:agent-alias/MLCUVGISY6/HLRZTEQL3T',
  'sub_agent_instruction': 'Lookup Sport event name or venu information and the athlete',
  'sub_agent_association_name': 'SportAgent',
  'relay_conversation_history': 'TO_COLLABORA

Associate all sub-agents with this one (main one).

In [231]:
media_ops_agent_alias_id, media_ops_agent_alias_arn = agents.associate_sub_agents(
    media_ops_agent_id, sub_agents_list
)

Waiting for agent status to change. Current status PREPARING
Agent id AD9HMFOUHU current status: PREPARED
Waiting for agent status to change. Current status PREPARING
Agent id AD9HMFOUHU current status: PREPARED
Waiting for agent status to change. Current status PREPARING
Agent id AD9HMFOUHU current status: PREPARED
Waiting for agent status to change. Current status PREPARING
Agent id AD9HMFOUHU current status: PREPARED


Check response return, to see all returned parameteres:

In [232]:
%store media_ops_agent_alias_id
%store media_ops_agent_alias_arn

Stored 'media_ops_agent_alias_id' (str)
Stored 'media_ops_agent_alias_arn' (str)


In [233]:
%store -r
print("Supervisor agent name:", agent_name, "id:", media_ops_agent_id)
print("Synopsis agent name:", syn_agent_name, "id:", syn_agent_id, "alias_id:", syn_agent_alias_id)
print("QC agent name:", qc_agent_name, "id:", qc_agent_id, "alias_id:", qc_agent_alias_id)
print("Film agent name:", film_agent_name, "id:", film_agent_id, "alias_id:", film_agent_alias_id)
print("Sport agent name:", sport_agent_name, "id:", sport_agent_id, "alias_id:", sport_agent_alias_id)

Supervisor agent name: su-agent-us-west-2-241 id: AD9HMFOUHU
Synopsis agent name: syn-agent-us-west-2-241 id: Q35CY7LJXH alias_id: WDJVWHIQ3X
QC agent name: qc-agent-us-west-2-241 id: 3C2MWJKT0D alias_id: IZ9Y83APAP
Film agent name: f-agent-us-west-2-241 id: YMRXSPKGV0 alias_id: C185HNE8PM
Sport agent name: s-agent-us-west-2-241 id: MLCUVGISY6 alias_id: HLRZTEQL3T


We can now set a dictionary with agents names for better visualization of the traces

In [234]:
multi_agent_names = {
    f"{media_ops_agent_id}/{media_ops_agent_alias_id}": agent_name,
    f"{syn_agent_id}/{syn_agent_alias_id}": syn_agent_name,
    f"{qc_agent_id}/{qc_agent_alias_id}": qc_agent_name,
    f"{film_agent_id}/{film_agent_alias_id}": film_agent_name,
    f"{sport_agent_id}/{sport_agent_alias_id}": sport_agent_name
}

## Create and syncronize Knowledge Base

On this section, you're going to create a Amazon Bedrock Knowledge Base and ingest data on it.

This data contains basic information about how forecast process is done.

**This creation process can take several minutes.**

In [235]:
%%time
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    bucket_name
)

print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {ds_id}")

Creating KB su-agent-us-west-2-241
Step 1 - Creating or retrieving su-agent-us-west-2-241-us-west-2-241533163649 S3 bucket for Knowledge Base documents
Creating bucket su-agent-us-west-2-241-us-west-2-241533163649
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_867) and Policies
Step 3 - Creating OSS encryption, network and data access policies
Step 4 - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '318',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Fri, 21 Mar 2025 13:27:49 '
                                                 'GMT',
                                         'x-amzn-requestid': '87fe5a56-b9cb-42fb-a799-7203b9f4afb9'},
                        'HTTPStatusCode': 200,
                     

### Upload Standard Documents to S3

In [236]:
!rm -rf `find -type d -name .ipynb_checkpoints`

In [237]:
standards_folder = "standards"

!aws s3 sync {standards_folder} s3://{bucket_name}/

upload: standards/logging.txt to s3://su-agent-us-west-2-241-us-west-2-241533163649/logging.txt
upload: standards/website.txt to s3://su-agent-us-west-2-241-us-west-2-241533163649/website.txt
upload: standards/logging.txt.metadata.json to s3://su-agent-us-west-2-241-us-west-2-241533163649/logging.txt.metadata.json
upload: standards/social.txt to s3://su-agent-us-west-2-241-us-west-2-241533163649/social.txt
upload: standards/social.txt.metadata.json to s3://su-agent-us-west-2-241-us-west-2-241533163649/social.txt.metadata.json
upload: standards/website.txt.metadata.json to s3://su-agent-us-west-2-241-us-west-2-241533163649/website.txt.metadata.json


In [238]:
# sync knowledge base
kb.synchronize_data(kb_id, ds_id)

{ 'dataSourceId': 'UZT6CCU2GN',
  'ingestionJobId': 'OUCHOCWUTV',
  'knowledgeBaseId': 'NHWQOW6QCP',
  'startedAt': datetime.datetime(2025, 3, 21, 13, 31, 23, 594655, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 0,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 0},
  'status': 'STARTING',
  'updatedAt': datetime.datetime(2025, 3, 21, 13, 31, 23, 594655, tzinfo=tzlocal())}
{ 'dataSourceId': 'UZT6CCU2GN',
  'ingestionJobId': 'OUCHOCWUTV',
  'knowledgeBaseId': 'NHWQOW6QCP',
  'startedAt': datetime.datetime(2025, 3, 21, 13, 31, 23, 594655, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 3,
  

In [239]:
kb_info = kb.get_kb(kb_id)
kb_arn = kb_info['knowledgeBase']['knowledgeBaseArn']

In [240]:
kb_config = {
    'kb_id': kb_id,
    'kb_instruction': """Use this knowledge base when you need to look up the guideline requirements and output schema (AS IS, NO paraphrasing or summary) for video post"""
}

Store environment variables to be used on next notebooks.

### Associating knowledge base
Now that we've created the agent, let's associate the previously created knowledge base to it.

In [241]:
agents.associate_kb_with_agent(
    media_ops_agent[0],
    kb_config['kb_instruction'],
    kb_config['kb_id']
)
time.sleep(20)

In [242]:
session_state = {
    'knowledgeBaseConfigurations': [
        {
            'knowledgeBaseId': kb_id,
            'retrievalConfiguration': {
                'vectorSearchConfiguration': {
                    'implicitFilterConfiguration': {
                        'metadataAttributes': [
                            {
                                'description': """Choose one base on the definition: 
                                - LOGGING: internal system logs
                                - SOCIAL: social media post
                                - WEBSITE: company website""",
                                'key': 'tag_type',
                                'type': 'STRING'
                            },
                        ],
                        'modelArn': "anthropic.claude-3-5-sonnet-20241022-v2:0"
                    },
                    'numberOfResults': 1,
                    'overrideSearchType': 'HYBRID'
                }
            }
        },
    ]
}

In [243]:
%%time
session_id:str = str(uuid.uuid4())

response = agents.invoke(
    "extract the output schema for social media post",
    media_ops_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names,
    session_state=session_state
)
print("====================")
print(response)

invokeAgent API request ID: ba5145cb-c1d8-4305-bf31-a618e57256ce
invokeAgent API session ID: 35c03e12-2755-4136-8e4c-f77ada727fe6
  agent id: AD9HMFOUHU, agent alias id: TSTALIASID
---- Step 1 ----
Took 5.3s, using 1844 tokens (in: 1736, out: 108) to complete prior action, observe, orchestrate.
I'll search the knowledge base to get the output schema requirements for social media posts.
Using knowledge base id: NHWQOW6QCP to search for:
  What is the output schema for social media post?

Knowledge base lookup output, 0 references:

---- Step 2 ----
Took 6.7s, using 130 tokens (in: 0, out: 130) to complete prior action, observe, orchestrate.
---- Step 3 ----
Took 3.5s, using 2037 tokens (in: 1932, out: 105) to complete prior action, observe, orchestrate.
Let me try a different search query to find the output schema requirements.
Using knowledge base id: NHWQOW6QCP to search for:
  What are the required fields and format for social media posts?

Knowledge base lookup output, 0 references:

On following section, you will invoke sub-agents but by multi-agent collaborator.

### Test Agent

In [255]:
video_s3_path = sport_video_s3_path
video_s3_path

's3://sagemaker-us-west-2-241533163649/media-operations-agent-claude/Grand-Prinx-Clip01.mp4'

In [256]:
video_analysis = sport_video_analysis
video_analysis

{'visual_summary': 'The video showcases a high-speed motorsports event, likely a Formula 1 race. It opens with close-up shots of a sleek, orange racing car and two race car drivers wearing white uniforms with the "Chandon" logo, standing on a racetrack. The scene then transitions to various angles of the race cars in motion, including a McLaren vehicle with the "McLaren Honda" branding and a yellow car bearing the "JOHNNIE WALKER" logo. The footage captures the intense atmosphere of the race, with blurred shots of the cars speeding down the track and pit crews working on the vehicles. Prominent logos and branding from sponsors like "Repsol," "Kenwood," and "Richard Mille" are visible throughout. The video also features glimpses of a racing circuit map, control rooms, and pit areas, providing a comprehensive look at the behind-the-scenes operations of a professional motorsports event.',
 'audio_transcription': ' ',
 'metadata': {'asset_id': '0',
  'semantic_modality': 'VIDEO',
  's3_buc

In [257]:
session_state = {
    'promptSessionAttributes': {
        "<video_analysis>": json.dumps(video_analysis),
        "<video_s3_path>": video_s3_path,
    },
    'knowledgeBaseConfigurations': [
        {
            'knowledgeBaseId': kb_id,
            'retrievalConfiguration': {
                'vectorSearchConfiguration': {
                    'implicitFilterConfiguration': {
                        'metadataAttributes': [
                            {
                                'description': """Choose one base on the definition: 
                                - LOGGING: internal system logs
                                - SOCIAL: social media post
                                - WEBSITE: company website""",
                                'key': 'tag_type',
                                'type': 'STRING'
                            },
                        ],
                        'modelArn': "anthropic.claude-3-5-sonnet-20241022-v2:0"
                    },
                    'numberOfResults': 1,
                    'overrideSearchType': 'HYBRID'
                }
            }
        },
    ]
}

In [271]:
import random

possible_post_types = ["social media", "company website", "internal logging system"]
video_types = ["Film", "Sports"]
post_type = random.choice(possible_post_types)

print(f"Create a compliant post for ## {post_type} ##")

Create a compliant post for ## internal logging system ##


In [272]:
prompt = f"""
    Follow the guideline requirements and expected output schema, create a compliant post for {post_type}.
    
    <video_extraction>
    $prompt_session_attributes.video_analysis$
    </video_extraction>

    <video_s3_path>
    $prompt_session_attributes.video_s3_path$
    </video_s3_path>

    Here is the instruction.
    <instruction>
    1. identify the type of the video: {str(video_types)}
    2. based on the video type, Write out each requirements <requirements> and the exact output schema <output_schema>. 
        - DO NOT make up new requirements. 
        - PAY ATTENTION to the word count requirement.
    3. If you need additional info, use <video_extraction> and video at <video_s3_path> to lookup info.
    4. then use <video_extraction> and additional info to create a post, make sure to following all the requirements.
    5. check the post against the <requirements> and output schema. Only report major violations. 
    6. If FAIL, improve the post to address the issues.
    7. repeat step 5 and 6 until the post PASS quality check.
    8. Providing the final output in the specified schema format, skip any explanation
    </instruction>
    """

In [273]:
%%time
session_id:str = str(uuid.uuid4())

response = agents.invoke(
    prompt,
    media_ops_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names,
    session_state=session_state
)
print("====================")
print(response)

invokeAgent API request ID: e6d6d138-319a-4daf-b028-74c45a9f0548
invokeAgent API session ID: 8ee1906e-4bfe-4d4f-aeac-bd7f279cbd6b
  agent id: AD9HMFOUHU, agent alias id: TSTALIASID
---- Step 1 ----
Took 6.0s, using 2694 tokens (in: 2526, out: 168) to complete prior action, observe, orchestrate.
Let me analyze this step by step:

1. First, I need to identify the video type from the video extraction. Based on the visual summary mentioning "Formula 1 race", "racing cars", "McLaren Honda", this is clearly a Sports video.

2. Let me look up the requirements and schema for Sports video internal logging post.
Using knowledge base id: NHWQOW6QCP to search for:
  What are the requirements and output schema for Sports video internal logging post?

Knowledge base lookup output, 1 references:

  (1) **Document Title:** Internal Logging Standards For Video Synopsis
**Department:** Technical Operations
**Last Updated:** March 2025
**Document Number:** LOG-INT-001

## Internal Logging Requirements

#

Generated code                                                                                                     

                                                                                                                   
 description = "Comprehensive footage capturing McLaren Honda's Formula 1 operations during the Bahrain Grand Prix 
 The video showcases the team's orange racing car along with two drivers in white uniforms displaying Chandon      
 branding. Multiple racing sequences feature the McLaren Honda car and other competitors, including detailed shots 
 of high-speed track action. Behind-the-scenes content includes pit crew operations, control room activities, and  
 circuit map analysis. Notable sponsor visibility includes Chandon, Johnnie Walker, Repsol, Kenwood, and Richard   
 Mille throughout various segments of the footage."                                                                
                                                                                                                   
 word_count = len(description.split())                                                                             
 print(f"Word count: {word_count}")                                                                                
                                                                                                                   

---- Step 4.2 [using sub-agent name:qc-agent-us-west-2-241, id:3C2MWJKT0D/IZ9Y83APAP] ----
Took 11.2s, using 1779 tokens (in: 1521, out: 258) to complete prior action, observe, orchestrate.
Let me evaluate against the requirements:
1. Description length: PASS (80 words, within 50-100 range)
2. Header format: PASS (clear and descriptive)
3. Required fields present: FAIL (missing date/year)
4. Technical specs: PASS (all required fields present)
5. Location/Venue: PASS (properly specified)
6. Video type: PASS (correctly categorized)
Final response:
{
    "status": "FAIL",
    "violations": [
        "Missing required field: date/year of the event",
        "Technical specs frame rate should be rounded to 2 decimal places (26.19 should be 26.20)"
    ],
    "review_notes": {
        "description_length": "PASS - 80 words",
        "required_fie...

----sub-agent QCAgent output text:
{
    "status": "FAIL",
    "violations": [
        "Missing required field: date/year of the event",
      

Generated code                                                                                                     

                                                                                                                   
 description = "Comprehensive footage capturing McLaren Honda's Formula 1 operations during the Bahrain Grand Prix 
 on March 2, 2024. The video showcases the team's orange racing car along with two drivers in white uniforms       
 displaying Chandon branding. Multiple racing sequences feature the McLaren Honda car and other competitors,       
 including detailed shots of high-speed track action. Behind-the-scenes content includes pit crew operations,      
 control room activities, and circuit map analysis. Notable sponsor visibility includes Chandon, Johnnie Walker,   
 Repsol, Kenwood, and Richard Mille throughout various segments of the footage."                                   
                                                                                                                   
 word_count = len(description.split())                                                                             
 print(f"Word count: {word_count}")                                                                                
                                                                                                                   

---- Step 5.2 [using sub-agent name:qc-agent-us-west-2-241, id:3C2MWJKT0D/IZ9Y83APAP] ----
Took 5.7s, using 2500 tokens (in: 2290, out: 210) to complete prior action, observe, orchestrate.
Let me evaluate the updated post against the requirements:
1. Description length: PASS (84 words, within 50-100 range)
2. Header format: PASS (clear and descriptive, includes year)
3. Required fields present: PASS (date added in both header and description)
4. Technical specs: PASS (all required fields present with correct format)
5. Location/Venue: PASS (properly specified)
6. Video type: PASS (correctly categorized)
Final response:
{
    "status": "PASS",
    "review_notes": {
        "description_length": "PASS - 84 words",
        "required_fields": "PASS - all present",
        "technical_specs": "PASS - correct format",
        "metadata_format": "PASS",
        "content_structure": "PASS"
    }
}...

----sub-agent QCAgent output text:
{
    "status": "PASS",
    "review_notes": {
        "desc

### Clean Up

In [219]:
# agents.delete_agent(agent_name)

Deleting alias QM32XSH3RJ from agent 6ALET6EZQZ
Deleting alias TSTALIASID from agent 6ALET6EZQZ


In [220]:
# kb.delete_kb(knowledge_base_name)

Data Source deleted successfully!
Knowledge Base deleted successfully!
OpenSource Serveless Index deleted successfully!
OpenSource Collection Index deleted successfully!
OpenSource Serveless access policy deleted successfully!
OpenSource Serveless network policy deleted successfully!
OpenSource Serveless encryption policy deleted successfully!
Knowledge Base S3 bucket deleted successfully!
Knowledge Base Roles and Policies deleted successfully!
Resources deleted successfully!
